# PhotometryLib

## Output directories

In [ ]:
import os
outdir = os.path.join(os.environ["HOME"], "notebook_output")
csvdir = os.path.join(outdir, "csv")
pngdir = os.path.join(outdir, "png")
! mkdir -p $csvdir                          
! mkdir -p $pngdir                          
! ls $outdir

## Importing dependencies

In [ ]:
from pathlib import Path
from astropy.io import fits
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
%matplotlib widget

datapath = Path("/data01/home/baroncelli/phd/DATA")
os.environ["DATA"] = str(datapath)
from RTAscience.cfg.Config import Config
from RTAscience.lib.RTAUtils import get_pointing
from rtapipe.lib.datasource.photometry import Photometry
from rtapipe.lib.plotting.photometryplot import PhotometrySinglePlot, PhotometrySubPlots

## Utility function

In [ ]:
from os import listdir
from os.path import isfile, join

def getInput(dataDir, index):
    
    if index == 0:
        index = 1

    simFolder = datapath.joinpath("obs",dataDir)
    cfg = Config(simFolder.joinpath("config.yaml"))
    runid = cfg.get('runid')
    template =  os.path.join(datapath, f'templates/{runid}.fits')
    pointing = get_pointing(template)
    
    
    if cfg.get("simtype") == 'bkg':
        dir = simFolder.joinpath("backgrounds")
    else:
        dir = simFolder.joinpath(runid)
                                                        
    data_files = [join(dir, f) for f in listdir(dir) if isfile(join(dir, f))]  

    output = []
    for data_file in data_files:
        print("datafile: ", data_file)
        conf = {
            'input_file': data_file,
            'output_dir': outdir,
            'simtype' : cfg.get('simtype'),
            'runid' : cfg.get('runid'),
            't_window_start': 0,
            't_window_stop': cfg.get('tobs'),
            'e_window_start': cfg.get('emin'),
            'e_window_stop': cfg.get('emax'),
            'onset' : cfg.get('onset')
        }
        
        output.append((data_file, conf, pointing))
        #inputFitsFile = str(f"ebl{str(index).zfill(6)}.fits"))
        #inputFitsFile = str(,f"bkg{str(index).zfill(6)}.fits"))

    print(f"Found: {len(output)} files, pointing is: {pointing}")
    return output


## Data folders (simulations)
    folder/
        config.yaml
        backgrounds/
            bkg000001.fits         => simulation background only
            ..
            bkg00000<N>.fits
        <runId>/
            ebl000001.fits         => simulation background + source
            ..
            ebl000001<N>.fits
            

## How to get data

### Data directories
Simulations from template: run0406_ID000126

In [ ]:
bkgsrc      = "simtype_grb_os_1800_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5"
bkgonly     = "simtype_bkg_os_0_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5"

### Get csv input, simulation parameters and pointing (from template)

In [ ]:
inputFile_srcbkg_0, sim_params_bkgsrc, pointing_srcbkg_0 = getInput(bkgsrc, 1)

In [ ]:
inputFile_onlybkg_0, sim_params_bkgonly, pointing_onlybkg_0 = getInput(bkgonly, 1)

### Differences between configuration dictionaries

In [ ]:
assert pointing_srcbkg_0 == pointing_onlybkg_0
assert inputFile_srcbkg_0 != inputFile_onlybkg_0
for key, val in sim_params_bkgsrc.items():
    if val!=sim_params_bkgonly[key]:
        print(f"{key}:\n\t{val}\n\t{sim_params_bkgonly[key]}")

## The photometry class

In [ ]:
photometry = Photometry()

## Photometry parameters

In [ ]:
photometry_params = {
    
    # integration parameters for time
    't_window_size': 25,
    't_window_step': 25,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : pointing_srcbkg_0,
    'region_radius': 0.5,
    
    # Other settings
    'plot' : 1,
    'override': 1
}

## Generating some data

### Integrate on time 

In [ ]:
csv_data_t_bkg_src = photometry.integrate(photometry_params, sim_params_bkgsrc, integration="t").pop(0)
#csv_data_t_bkg_src = "/data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits"
pd.read_csv(csv_data_t_bkg_src).head(3)

In [ ]:
csv_data_t_bkg_only = photometry.integrate(photometry_params, sim_params_bkgonly, integration="t").pop(0)
# csv_data_t_bkg_only = "/data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits"
print(pd.read_csv(csv_data_t_bkg_only).head(3))

### Integrate on energy

In [ ]:
csv_data_e_bkg_src = photometry.integrate(photometry_params, sim_params_bkgsrc, integration="e").pop(0)

In [ ]:
csv_data_e_bkg_only = photometry.integrate(photometry_params, sim_params_bkgonly, integration="e").pop(0)

## Plotting the data

### Time integration

In [ ]:
singleplot_t = PhotometrySinglePlot(title = "Time integration")
_ = singleplot_t.addData(csv_data_t_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "t")
_ = singleplot_t.addData(csv_data_t_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "t")

In [ ]:
multiplot_t = PhotometrySubPlots(title = "Time integration")
_ = multiplot_t.addData(csv_data_t_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "t")
_ = multiplot_t.addData(csv_data_t_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "t")

## Energy integration

In [ ]:
singleplot_e = PhotometrySinglePlot(title = "Energy integration")
singleplot_e.addData(csv_data_e_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "e")
singleplot_e.addData(csv_data_e_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "e")

In [ ]:
multiplot_e = PhotometrySubPlots(title = "Energy integration")
_ = multiplot_e.addData(csv_data_e_bkg_src, photometry_params, sim_params_bkgsrc, ["bkg+src"], "e")
_ = multiplot_e.addData(csv_data_e_bkg_only, photometry_params, sim_params_bkgonly, ["bkg only"], "e")

## Integration on time with multiple window sizes and background baseline

In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title="Integration on time with multiple window sizes")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title="Integration on time with multiple window sizes")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots(title="Integration on time with multiple window sizes and background baseline")

photometry_params_tmp = photometry_params.copy()

for ws in [10, 25, 50, 75]:

    # source+background
    photometry_params_tmp["t_window_size"] = ws
    photometry_params_tmp["t_window_step"] = ws 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="t").pop(0)
    
    
    # background only baseline
    photometry_params_tmp["t_window_size"] = ws
    photometry_params_tmp["t_window_step"] = ws
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="t").pop(0)
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["t_window_size","t_window_step"], "t")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["t_window_size","t_window_step"], "t")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["t_window_size","t_window_step"], "t")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["baseline (only background)"], "t", as_baseline=True)
    

In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_ws_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_ws_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_ws_bgksrc"))
multiplot_bgksrc.destroy()

## Integration on energy with multiple window sizes and background baseline

In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title = "Integration on energy with multiple window sizes")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title = "Integration on energy with multiple window sizes")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots(title = "Integration on energy with multiple window sizes and background baseline")

photometry_params_tmp = photometry_params.copy()

for ee in [0.001, 0.01, 0.02, 0.03]:

    # source+background
    photometry_params_tmp["e_window_size"] = ee
    photometry_params_tmp["e_window_step"] = ee 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="e").pop(0)
    
    
    # background only baseline
    photometry_params_tmp["e_window_size"] = ee
    photometry_params_tmp["e_window_step"] = ee
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="e").pop(0)
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["e_window_size","e_window_step"], "e")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["e_window_size","e_window_step"], "e")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["t_window_size","t_window_step"], "e")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["t_window_size","t_window_step"], "e", as_baseline=True)
    

In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_e_ws_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_e_ws_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_e_ws_bgksrc"))
multiplot_bgksrc.destroy()

## Integration on time with multiple region radius values and background baseline

In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title="Integration on time with multiple region radius values")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title="Integration on time with multiple region radius values")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots(title="Integration on time with multiple region radius values and background baseline")

photometry_params_tmp = photometry_params.copy()

for rr in [0.1, 1.0, 2.0, 3.0]:

    # source+background
    photometry_params_tmp["region_radius"] = rr
    photometry_params_tmp["region_radius"] = rr 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="t").pop(0)
    
    
    # background only baseline
    photometry_params_tmp["region_radius"] = rr
    photometry_params_tmp["region_radius"] = rr
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="t").pop(0)
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["region_radius"], "t")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["region_radius"], "t")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["region_radius"], "t")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["region_radius"], "t", as_baseline=True)
    

In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_rr_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_rr_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_rr_bgksrc"))
multiplot_bgksrc.destroy()

## Integration on time with multiple pointing values and baseline on original pointing

In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot(title="Integration on time with multiple pointing values")

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot(title="Integration on time with multiple pointing values")

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc_1 = PhotometrySubPlots(title="Integration on time with multiple pointing values and baseline on background")

# fouth plot: background + source on multiple window sizes (separate subplots with original pointing baseline)
multiplot_bgksrc_2 = PhotometrySubPlots(title="Integration on time with multiple pointing values and baseline on original pointing")


photometry_params_tmp = photometry_params.copy()
pointing=pointing_srcbkg_0
pointing_offset = 0.5
pointings = [
    (pointing_srcbkg_0[0]+pointing_offset,pointing[1]), 
    (pointing[0],pointing[1]+pointing_offset), 
    (pointing[0]-pointing_offset,pointing[1]), 
    (pointing[0],pointing[1]-pointing_offset)
] 

photometry_params_tmp2 = photometry_params.copy()
csv_data_t_orig_point = photometry.integrate(photometry_params_tmp2, sim_params_bkgsrc, integration="t").pop(0)

for pp in pointings:

    # source+background
    photometry_params_tmp["pointing"] = pp
    photometry_params_tmp["pointing"] = pp 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc, integration="t").pop(0)
    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["pointing"], "t")

    # background only
    photometry_params_tmp["pointing"] = pp
    photometry_params_tmp["pointing"] = pp
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, sim_params_bkgonly, integration="t").pop(0)
    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, ["pointing"], "t")
    
    # source+background + baseline background only
    _ = multiplot_bgksrc_1.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["pointing"], "t")
    _ = multiplot_bgksrc_1.addData(csv_data_t_bkg, photometry_params_tmp, sim_params_bkgonly, [f"baseline (background only)"], "t", as_baseline=True)    
    
    """
        @FIXME !!!
    """
    # source+background + baseline original pointing
    _ = multiplot_bgksrc_2.addData(csv_data_t_bkg_src, photometry_params_tmp, sim_params_bkgsrc, ["pointing"], "t")
    _ = multiplot_bgksrc_2.addData(csv_data_t_orig_point, photometry_params_tmp2, sim_params_bkgsrc, [f"baseline {pointing}"], "t", as_baseline=True)

print(f"Original pointing: {pointing_srcbkg_0}")


In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_p_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_p_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_p_bgksrc"))
multiplot_bgksrc.destroy()

## Plotting different trials

### bkg + src

In [ ]:
bkg_src_files = []
for i in range(4):
    bkg_src_files.append(getInput(bkgsrc, i))

In [ ]:
singleplot = PhotometrySinglePlot(title="Multiple trials")
multiplot = PhotometrySubPlots(title="Multiple trials")

for i in range(4):
    conf = bkg_src_files[i][1]
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)
    _ = singleplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")
    _ = multiplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")
    

### Bkg only

In [ ]:
bkg_only_files = []
for i in range(4):
    bkg_only_files.append(getInput(bkgonly, i))

In [ ]:
singleplot = PhotometrySinglePlot(title="Multiple trials")
multiplot = PhotometrySubPlots(title="Multiple trials")

for i in range(4):
    conf = bkg_only_files[i][1]
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)
    _ = singleplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")
    _ = multiplot.addData(csv_data_t, photometry_params, conf, ["pointing"], "t")

## Integrate on time and on energy

In [ ]:
sim_params_bkgsrc_tmp = sim_params_bkgsrc.copy()
sim_params_bkgsrc_tmp["t_window_start"] = 1000 

photometry_params_tmp = photometry_params.copy()
photometry_params_tmp["t_window_size"] = 50
photometry_params_tmp["t_window_step"] = 50
photometry_params_tmp["e_window_size"] = 0.01
photometry_params_tmp["e_window_step"] = 0.01

In [ ]:
csv_files = photometry.integrate(photometry_params_tmp, sim_params_bkgsrc_tmp, integration="et")

In [ ]:
csv_files_to_plot = [csv_files[0], csv_files[3], csv_files[-4], csv_files[-1]]
csv_files_to_plot

In [ ]:
def exctractInterval(f):
    emin,emax = f.split("energy_window_")[1].split(".csv")[0].split("_")
    return emin,emax

In [ ]:
singleplot_t = PhotometrySinglePlot(title = "Energy/Time integration")
for f in csv_files_to_plot:
    emin, emax = exctractInterval(f)
    _ = singleplot_t.addData(f, photometry_params_tmp, sim_params_bkgsrc_tmp, [f"emin: {emin} emax: {emax}"], "t")

## Generate data for an Encoder/Decoder LSTM

In [1]:
import os
outdir = os.path.join(os.environ["HOME"], "notebook_output")
csvdir = os.path.join(outdir, "csv")
pngdir = os.path.join(outdir, "png")
! mkdir -p $csvdir                          
! mkdir -p $pngdir                          
! ls $outdir

from pathlib import Path
from astropy.io import fits
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
%matplotlib widget

datapath = Path("/data01/home/baroncelli/phd/DATA")
os.environ["DATA"] = str(datapath)
from RTAscience.cfg.Config import Config
from RTAscience.lib.RTAUtils import get_pointing
from rtapipe.lib.datasource.photometry import Photometry
from rtapipe.lib.plotting.photometryplot import PhotometrySinglePlot, PhotometrySubPlots

csv  png



Bad key "text.kerning_factor" on line 4 in
/data01/home/baroncelli/.conda/envs/bphd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
photometry = Photometry()

photometry_params = {
    
    # integration parameters for time
    't_window_size': 25,
    't_window_step': 25,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : None,
    'region_radius': 0.5,
    
    # Other settings
    'plot' : 1,
    'override': 1
}

from os import listdir
from os.path import isfile, join

def getInput(dataDir, howMany=-1):

    simFolder = datapath.joinpath("obs", dataDir)
    cfg = Config(simFolder.joinpath("config.yaml"))
    runid = cfg.get('runid')
    template =  os.path.join(datapath, f'templates/{runid}.fits')
    pointing = get_pointing(template)
    
    
    #if cfg.get("simtype") == 'bkg':
    #    dir = simFolder.joinpath("backgrounds")
    #else:
    #    dir = simFolder.joinpath(runid)
                                                        
    data_files = [join(simFolder, f) for f in listdir(simFolder) if isfile(join(simFolder, f)) and ".yaml" not in f]  

    files = []
    for data_file in data_files:

        conf = {
            'input_file': data_file,
            'output_dir': outdir,
            'simtype' : cfg.get('simtype'),
            'runid' : cfg.get('runid'),
            't_window_start': 0,
            't_window_stop': cfg.get('tobs'),
            'e_window_start': cfg.get('emin'),
            'e_window_stop': cfg.get('emax'),
            'onset' : cfg.get('onset')
        }
        
        files.append(data_file)

        if len(files) >= howMany:
            break
            
            
    print(f"Found: {len(files)} files, pointing is: {pointing}")

    return conf, pointing, files


## Background only 

In [3]:
bkgonly = "run0406_ID000126_OK_bkg_only"

In [4]:
conf, pointing, files = getInput(bkgonly, 100)

Found: 100 files, pointing is: (33.057, -51.841)


In [5]:
photometry_params['pointing'] = pointing

In [6]:
#singleplot = PhotometrySinglePlot(title="Multiple trials")

for i in range(1,11):
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)    
    #_ = singleplot.addData(csv_data_t, photometry_params, conf, ["bkg"], "t")


  0%|          | 0/72 [00:00<?, ?it/s]

region_count: 0, rr: 0.5, tmin=0 tmax=25 emin=0.03 emax=0.15
..


  1%|▏         | 1/72 [00:01<01:57,  1.65s/it]

region_count: 0, rr: 0.5, tmin=25 tmax=50 emin=0.03 emax=0.15
..


  3%|▎         | 2/72 [00:02<01:15,  1.08s/it]

...

region_count: 0, rr: 0.5, tmin=150 tmax=175 emin=0.03 emax=0.15
..


 10%|▉         | 7/72 [00:05<00:39,  1.63it/s]

region_count: 0, rr: 0.5, tmin=175 tmax=200 emin=0.03 emax=0.15


 10%|▉         | 7/72 [00:17<02:44,  2.53s/it]


KeyboardInterrupt: Interrupted by user

## GRB with offset

In [ ]:
bkg_bkgsrc  = "simtype_grb_os_5000_tobs_18000_irf_South_z40_average_LST_5h_emin_0.03_emax_0.15_roi_2.5"

In [ ]:
inputs = getInput(bkg_bkgsrc, -1)

photometry_params['pointing'] = inputs[0][2]

print("photometry_params pointing: ",photometry_params['pointing'])
for idx, inp in enumerate(inputs):
    conf = inp[1]
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)    

    

## GRB data

In [ ]:
bkgsrc      = "simtype_grb_os_0_tobs_3600_irf_South_z40_average_LST_30m_emin_0.03_emax_0.15_roi_2.5"

In [ ]:
FILES_N = 100

bkgsrc_files = []
for i in range(FILES_N):
    bkgsrc_files.append(getInput(bkgsrc, i))

photometry_params['pointing'] = bkgsrc_files[0][2]
    
for i in range(FILES_N):
    conf = bkgsrc_files[i][1]
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)    

In [ ]:
singleplot = PhotometrySinglePlot(title="Multiple trials")

for i in range(4):
    conf = bkgsrc_files[i][1]
    csv_data_t = photometry.integrate(photometry_params, conf, integration="t").pop(0)
    _ = singleplot.addData(csv_data_t, photometry_params, conf, ["bkg"], "t")


# TODO

* fix simulation outdir name (it should include other params) (done)
* onset moved into template params (done)
* Geneare background più corti (done)
* custom titles (done)
* refactoring: own classes, getTitle() and getLabel() in base class (done)
* plottare differenti trials (done)
* refactoring photometry (done)
* round a 4 decimali (done)
* et, te integrations (done)
* plottare et (done)

* bug pointing as baseline
* plottare differenti runid